In [1]:
import numpy as np
import os
import shutil
import subprocess



In [2]:
total_count = 48


replica_count = 4

n_clusters = int(total_count / replica_count)

nclients = n_clusters

In [3]:
for folder in os.listdir('.'):
    if 'config' in folder and folder!='config' and 'zip' not in folder and '.' not in folder:
        shutil.rmtree(folder)

In [4]:
os.system("aws --region ap-south-1 ec2 describe-instances --filter Name=instance-type,Values=t2.small --query 'Reservations[*].Instances[*].[PrivateIpAddress]' --output text > all_internal_ips")

os.system("aws --region ap-south-1 ec2 describe-instances  --filter Name=instance-type,Values=t2.small --query 'Reservations[*].Instances[*].[PublicIpAddress]' --output text > all_external_ips")


# os.system("aws --region us-west-2 ec2 describe-instances --filter Name=instance-type,Values=c4.4xlarge --query 'Reservations[*].Instances[*].[PrivateIpAddress]' --output text >> all_internal_ips")

# os.system("aws --region us-west-2 ec2 describe-instances  --filter Name=instance-type,Values=c4.4xlarge --query 'Reservations[*].Instances[*].[PublicIpAddress]' --output text >> all_external_ips")



# os.system("aws --region us-west-2 ec2 describe-instances  --query 'Reservations[*].Instances[*].[PrivateIpAddress]' --output text > all_internal_ips")

# os.system("aws --region us-west-2 ec2 describe-instances --query 'Reservations[*].Instances[*].[PublicIpAddress]' --output text > all_external_ips")


0

In [5]:
with open('all_internal_ips') as f:
    ipList = f.read().splitlines()
ipList = [x for x in ipList if x!='None']

In [6]:
ipList, len(ipList)

(['172.31.35.131',
  '172.31.40.250',
  '172.31.37.249',
  '172.31.43.251',
  '172.31.45.250',
  '172.31.46.67',
  '172.31.44.125',
  '172.31.40.71',
  '172.31.38.70',
  '172.31.37.227',
  '172.31.45.226',
  '172.31.47.245',
  '172.31.40.110',
  '172.31.40.118',
  '172.31.43.118',
  '172.31.36.248',
  '172.31.47.119',
  '172.31.44.158',
  '172.31.38.158',
  '172.31.39.31',
  '172.31.34.145',
  '172.31.39.14',
  '172.31.35.21',
  '172.31.33.145',
  '172.31.46.151',
  '172.31.44.151',
  '172.31.47.154',
  '172.31.45.24',
  '172.31.40.58',
  '172.31.35.57',
  '172.31.36.61',
  '172.31.32.187',
  '172.31.36.7',
  '172.31.46.7',
  '172.31.45.14',
  '172.31.40.140',
  '172.31.39.42',
  '172.31.37.169',
  '172.31.43.47',
  '172.31.34.175',
  '172.31.44.52',
  '172.31.43.180',
  '172.31.45.55',
  '172.31.43.181',
  '172.31.47.33',
  '172.31.47.160',
  '172.31.41.36',
  '172.31.33.162',
  '172.31.36.38',
  '172.31.43.164',
  '172.31.40.41',
  '172.31.40.38',
  '172.31.47.75',
  '172.31.46.73',


In [7]:
import numpy as np
import os
import shutil
import subprocess



def write_list_to_file(file_path, my_list):
    try:
        # Open the file in write mode ('w')
        with open(file_path, 'w') as file:
            # Loop through each element in the list
            for item in my_list:
                # Write the item followed by a newline character to separate items
                file.write(str(item) + '\n')
        print("List has been successfully written to the file.")
    except IOError:
        print("An error occurred while writing the file.")



nregions = 3

latency_map = np.zeros((nregions, nregions))

latency_map[0,1] = 6
latency_map[1,0] = 6


latency_map[0,2] = 11
latency_map[2,0] = 11

latency_map[2,1] = 7
latency_map[1,2] = 7


# 2 cluster map

node_cluster_map = {
    0:0,
    1:0,
    2:0,
    3:0,
    4:1,
    5:1,
    6:1,
    7:1,
    8:2,
    9:2,
    10:2,
    11:2,
    12:3,
    13:3,
    14:3,
    15:3,
    16:4,
    17:4,
    18:4,
    19:4,
    20:5,
    21:5,
    22:5,
    23:5,
    24:6,
    25:6,
    26:6,
    27:6,
    28:7,
    29:7,
    30:7,
    31:7,
    32:8,
    33:8,
    34:8,
    35:8,
    36:9,
    37:9,
    38:9,
    39:9,
    40:10,
    41:10,
    42:10,
    43:10,
    44:11,
    45:11,
    46:11,
    47:11,
    48:0,
    49:1,
    50:2,
    51:3,
    52:4,
    53:5,
    54:6,
    55:7,
    56:8,
    57:9,
    58:10,
    59:11,
    
}


with open('all_internal_ips') as f:
    ipList = f.read().splitlines()
ipList = [x for x in ipList if x!='None']

ipList, len(ipList)



os.system('cd simul_latency/; rm *')

for i, ip in enumerate(ipList):
    print(i,ip)
    
    Lines = []
    
    Lines.append('tc qdisc add dev eth0 root handle 1: htb')
    Lines.append('')
    
    
    count_ip = 1
    
    for  j, ip_loop in enumerate(ipList):
        if j!=i:
            
            cls_i = node_cluster_map[i]%nregions
            cls_j = node_cluster_map[j]%nregions
            
            latency_delay = latency_map[cls_i, cls_j]
            
            Lines.append('tc class add dev eth0 parent 1: classid 1:'+str(count_ip)+' htb rate 1000mbit')
            Lines.append('tc filter add dev eth0 parent 1: protocol ip prio '+str(count_ip)+' u32 flowid 1:'+str(count_ip)+' match ip dst '+ip_loop+'/32')
            Lines.append('tc qdisc add dev eth0 parent 1:'+str(count_ip)+' handle '+str(count_ip)+'0: netem delay '+str(latency_delay)+'ms')
            Lines.append('')
            
            
            count_ip = count_ip + 1

    
    
    write_list_to_file('simul_latency/node_'+str(i)+'.sh', Lines)
    
    
#     break






0 172.31.35.131
List has been successfully written to the file.
1 172.31.40.250
List has been successfully written to the file.
2 172.31.37.249
List has been successfully written to the file.
3 172.31.43.251
List has been successfully written to the file.
4 172.31.45.250
List has been successfully written to the file.
5 172.31.46.67
List has been successfully written to the file.
6 172.31.44.125
List has been successfully written to the file.
7 172.31.40.71
List has been successfully written to the file.
8 172.31.38.70
List has been successfully written to the file.
9 172.31.37.227
List has been successfully written to the file.
10 172.31.45.226
List has been successfully written to the file.
11 172.31.47.245
List has been successfully written to the file.
12 172.31.40.110
List has been successfully written to the file.
13 172.31.40.118
List has been successfully written to the file.
14 172.31.43.118
List has been successfully written to the file.
15 172.31.36.248
List has been success

In [8]:
clusterIpMap = {}

for i in range(len(ipList)):
    clusterIpMap[i] = ipList[i]

    
for i in range(nclients):
    
    clusterIpMap[7001+i] = ipList[-(nclients-i)]




clusterPortMap1 = {}
clusterPortMap2 = {}

for i in range(len(ipList) - nclients):
    
    clusterPortMap1[i] = str(11000+i*10)
    
for i in range(len(ipList) - nclients):
    clusterPortMap2[i] = str(11000+i*10+1)
    
# clusterPortMap1[7003] = '11400'
# clusterPortMap2[7003] = '11410'



temp = 11900

for i in range(nclients):
    
    clusterPortMap1[7001+i] = str(temp+i*20)
    clusterPortMap2[7001+i] = str(temp+i*20+10)



In [9]:
clusterPortMap1, clusterPortMap2

({0: '11000',
  1: '11010',
  2: '11020',
  3: '11030',
  4: '11040',
  5: '11050',
  6: '11060',
  7: '11070',
  8: '11080',
  9: '11090',
  10: '11100',
  11: '11110',
  12: '11120',
  13: '11130',
  14: '11140',
  15: '11150',
  16: '11160',
  17: '11170',
  18: '11180',
  19: '11190',
  20: '11200',
  21: '11210',
  22: '11220',
  23: '11230',
  24: '11240',
  25: '11250',
  26: '11260',
  27: '11270',
  28: '11280',
  29: '11290',
  30: '11300',
  31: '11310',
  32: '11320',
  33: '11330',
  34: '11340',
  35: '11350',
  36: '11360',
  37: '11370',
  38: '11380',
  39: '11390',
  40: '11400',
  41: '11410',
  42: '11420',
  43: '11430',
  44: '11440',
  45: '11450',
  46: '11460',
  47: '11470',
  7001: '11900',
  7002: '11920',
  7003: '11940',
  7004: '11960',
  7005: '11980',
  7006: '12000',
  7007: '12020',
  7008: '12040',
  7009: '12060',
  7010: '12080',
  7011: '12100',
  7012: '12120'},
 {0: '11001',
  1: '11011',
  2: '11021',
  3: '11031',
  4: '11041',
  5: '11051',
 

In [10]:
os.system('echo 3108 | sudo -S ./gradlew installDist')

[sudo] password for tejas: 

> Task :compileJava UP-TO-DATE
> Task :processResources NO-SOURCE
> Task :classes UP-TO-DATE
> Task :jar UP-TO-DATE
> Task :installDist

BUILD SUCCESSFUL in 1s
3 actionable tasks: 1 executed, 2 up-to-date


0

In [11]:
for i in range(total_count):
    os.system('./runscripts/smartrun.sh bftsmart.tom.util.RSAKeyPairGenerator '+str(i)+' 1024')

    

os.system('./runscripts/smartrun.sh bftsmart.tom.util.RSAKeyPairGenerator '+str(7002)+' 1024')
os.system('./runscripts/smartrun.sh bftsmart.tom.util.RSAKeyPairGenerator '+str(7001)+' 1024')

0

In [12]:
os.system('echo 3108 | sudo -S ./gradlew clean')

[sudo] password for tejas: 

> Task :clean

BUILD SUCCESSFUL in 971ms
1 actionable task: 1 executed


0

In [13]:
for key in clusterPortMap1.keys():
    if key>7002:
        print(key)
        print()
        
        for folder in os.listdir('config/'):
            if os.path.isdir('config/'+folder) and 'keys' in folder:
                if os.path.exists('config/'+folder+'/'+'privatekey7001'):
                    shutil.copy('config/'+folder+'/'+'privatekey7001','config/'+folder+'/'+'privatekey'+str(key))
                    shutil.copy('config/'+folder+'/'+'publickey7001','config/'+folder+'/'+'publickey'+str(key))
                
        print('----------')
        print()

7003

----------

7004

----------

7005

----------

7006

----------

7007

----------

7008

----------

7009

----------

7010

----------

7011

----------

7012

----------



In [14]:
for i in range(n_clusters):
    
    if os.path.exists('config'+str(i)):
        shutil.rmtree('config'+str(i))
    
    os.mkdir('config'+str(i))
    os.mkdir('config'+str(i)+'/keysRSA')
    os.mkdir('config'+str(i)+'/keysSunEC')
    os.mkdir('config'+str(i)+'/keysECDSA')
    
    
    shutil.copytree('config/workloads/', 'config'+str(i)+'/workloads')
    shutil.copytree('config/keysSSL_TLS/', 'config'+str(i)+'/keysSSL_TLS')
    
    
    for file in os.listdir('config/keys/'):
    
        shutil.copy('config/keys/'+file, 'config'+str(i)+'/keysRSA/'+file)
        shutil.copy('config/keys/'+file, 'config'+str(i)+'/keysSunEC/'+file)
        shutil.copy('config/keys/'+file, 'config'+str(i)+'/keysECDSA/'+file)
    
    
    shutil.copy('config/system.config', 'config'+str(i)+'/system.config')    
    shutil.copy('config/logback.xml', 'config'+str(i)+'/logback.xml')
    
    
    
    

In [15]:
        
        for i in range(nclients):
            print(str(7001+i)+' '+clusterIpMap[7001+i]+' '+clusterPortMap1[7001+i]+'\n')

7001 172.31.36.38 11900

7002 172.31.43.164 11920

7003 172.31.40.41 11940

7004 172.31.40.38 11960

7005 172.31.47.75 11980

7006 172.31.46.73 12000

7007 172.31.45.207 12020

7008 172.31.41.76 12040

7009 172.31.44.82 12060

7010 172.31.46.209 12080

7011 172.31.37.89 12100

7012 172.31.32.84 12120



In [16]:
for n_cluster in range(n_clusters):
    
    with open('config'+str(n_cluster)+'/hosts.config','w') as file:
        
        for n_cluster1 in range(n_clusters):
        
            for i_replica in range(replica_count):
    #             file.writelines(str(i_replica)+' '+clusterIpMap[n_cluster*replica_count+i_replica]+' '  +clusterPortMap1[n_cluster*replica_count+i_replica]+' ' +  clusterPortMap2[n_cluster*replica_count+i_replica]+'\n')
                file.writelines(str(n_cluster1*replica_count+i_replica)+' '+clusterIpMap[n_cluster1*replica_count+i_replica]+' '  +clusterPortMap1[n_cluster1*replica_count+i_replica]+' ' +  clusterPortMap2[n_cluster1*replica_count+i_replica]+' '+str(n_cluster1)+'\n')

        for iter_ in range(len(ipList) - (n_clusters*replica_count)-nclients):
            print((n_clusters*replica_count) + iter_)
            
#             file.writelines(str((n_clusters*replica_count) + iter_)+' '+clusterIpMap[(n_clusters*replica_count) + iter_]+' '  +clusterPortMap1[(n_clusters*replica_count) + iter_]+' ' +  clusterPortMap2[(n_clusters*replica_count) + iter_]+' '+str(iter_%n_clusters)+'\n')
            file.writelines(str((n_clusters*replica_count) + iter_)+' '+clusterIpMap[(n_clusters*replica_count) + iter_]+' '  +clusterPortMap1[(n_clusters*replica_count) + iter_]+' ' +  clusterPortMap2[(n_clusters*replica_count) + iter_]+' '+str(0%n_clusters)+'\n')
        
        for i in range(nclients):
            file.writelines(str(7001+i)+' '+clusterIpMap[7001+i]+' '+clusterPortMap1[7001+i]+'\n')

        
        file.close()


In [17]:
replica_count

4

In [18]:
for n_cluster in range(n_clusters):
    with open('config'+str(n_cluster)+'/system.config','r') as file:
        data = file.readlines()
        f = int(np.ceil((replica_count-1)/3))
        print('f is',f )
        
        for n_line in range(len(data)):
            if ('system.servers.num' in data[n_line]) and '#' not in data[n_line]:
                
                data[n_line] = 'system.servers.num = '+str(replica_count)
                
            if ('system.servers.f' in data[n_line]) and '#' not in data[n_line]:
                data[n_line] = 'system.servers.f = '+str(f)
                
            if ('system.initial.view' in data[n_line]) and '#' not in data[n_line]:
                data[n_line] = 'system.initial.view = '
                for iter_view in range(replica_count):
                    data[n_line] = data[n_line] +str(n_cluster*replica_count+iter_view)+','
#                     data[n_line] = data[n_line] +str(n_cluster*replica_count+iter_view)+','
                
                data[n_line] = data[n_line][:-1]
                
                
        file.close()
        
    with open('config'+str(n_cluster)+'/system.config', 'w') as f:
        for line in data:
            f.write(f"{line}\n")
        

f is 1
f is 1
f is 1
f is 1
f is 1
f is 1
f is 1
f is 1
f is 1
f is 1
f is 1
f is 1


In [19]:
for n_cluster in range(n_clusters):
    for i in range(400):
        
        for ncls in  range(1):
        
        
            print('config'+str(n_cluster)+'/keysRSA/'+'publickey7002', \
                        'config'+str(n_cluster)+'/keysRSA/'+'publickey'+str(1000+(ncls*100)+i))
            shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'publickey7002', \
                        'config'+str(n_cluster)+'/keysRSA/'+'publickey'+str(1000+(ncls*100)+i))
            shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'privatekey7002', \
                        'config'+str(n_cluster)+'/keysRSA/'+'privatekey'+str(1000+(ncls*100)+i))
        
#         shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'publickey7002', \
#                     'config'+str(n_cluster)+'/keysRSA/'+'publickey7002')
        
#         shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'privatekey7002', \
#                     'config'+str(n_cluster)+'/keysRSA/'+'privatekey7002')

config0/keysRSA/publickey7002 config0/keysRSA/publickey1000
config0/keysRSA/publickey7002 config0/keysRSA/publickey1001
config0/keysRSA/publickey7002 config0/keysRSA/publickey1002
config0/keysRSA/publickey7002 config0/keysRSA/publickey1003
config0/keysRSA/publickey7002 config0/keysRSA/publickey1004
config0/keysRSA/publickey7002 config0/keysRSA/publickey1005
config0/keysRSA/publickey7002 config0/keysRSA/publickey1006
config0/keysRSA/publickey7002 config0/keysRSA/publickey1007
config0/keysRSA/publickey7002 config0/keysRSA/publickey1008
config0/keysRSA/publickey7002 config0/keysRSA/publickey1009
config0/keysRSA/publickey7002 config0/keysRSA/publickey1010
config0/keysRSA/publickey7002 config0/keysRSA/publickey1011
config0/keysRSA/publickey7002 config0/keysRSA/publickey1012
config0/keysRSA/publickey7002 config0/keysRSA/publickey1013
config0/keysRSA/publickey7002 config0/keysRSA/publickey1014
config0/keysRSA/publickey7002 config0/keysRSA/publickey1015
config0/keysRSA/publickey7002 config0/ke

config0/keysRSA/publickey7002 config0/keysRSA/publickey1217
config0/keysRSA/publickey7002 config0/keysRSA/publickey1218
config0/keysRSA/publickey7002 config0/keysRSA/publickey1219
config0/keysRSA/publickey7002 config0/keysRSA/publickey1220
config0/keysRSA/publickey7002 config0/keysRSA/publickey1221
config0/keysRSA/publickey7002 config0/keysRSA/publickey1222
config0/keysRSA/publickey7002 config0/keysRSA/publickey1223
config0/keysRSA/publickey7002 config0/keysRSA/publickey1224
config0/keysRSA/publickey7002 config0/keysRSA/publickey1225
config0/keysRSA/publickey7002 config0/keysRSA/publickey1226
config0/keysRSA/publickey7002 config0/keysRSA/publickey1227
config0/keysRSA/publickey7002 config0/keysRSA/publickey1228
config0/keysRSA/publickey7002 config0/keysRSA/publickey1229
config0/keysRSA/publickey7002 config0/keysRSA/publickey1230
config0/keysRSA/publickey7002 config0/keysRSA/publickey1231
config0/keysRSA/publickey7002 config0/keysRSA/publickey1232
config0/keysRSA/publickey7002 config0/ke

config0/keysRSA/publickey7002 config0/keysRSA/publickey1358
config0/keysRSA/publickey7002 config0/keysRSA/publickey1359
config0/keysRSA/publickey7002 config0/keysRSA/publickey1360
config0/keysRSA/publickey7002 config0/keysRSA/publickey1361
config0/keysRSA/publickey7002 config0/keysRSA/publickey1362
config0/keysRSA/publickey7002 config0/keysRSA/publickey1363
config0/keysRSA/publickey7002 config0/keysRSA/publickey1364
config0/keysRSA/publickey7002 config0/keysRSA/publickey1365
config0/keysRSA/publickey7002 config0/keysRSA/publickey1366
config0/keysRSA/publickey7002 config0/keysRSA/publickey1367
config0/keysRSA/publickey7002 config0/keysRSA/publickey1368
config0/keysRSA/publickey7002 config0/keysRSA/publickey1369
config0/keysRSA/publickey7002 config0/keysRSA/publickey1370
config0/keysRSA/publickey7002 config0/keysRSA/publickey1371
config0/keysRSA/publickey7002 config0/keysRSA/publickey1372
config0/keysRSA/publickey7002 config0/keysRSA/publickey1373
config0/keysRSA/publickey7002 config0/ke

config1/keysRSA/publickey7002 config1/keysRSA/publickey1243
config1/keysRSA/publickey7002 config1/keysRSA/publickey1244
config1/keysRSA/publickey7002 config1/keysRSA/publickey1245
config1/keysRSA/publickey7002 config1/keysRSA/publickey1246
config1/keysRSA/publickey7002 config1/keysRSA/publickey1247
config1/keysRSA/publickey7002 config1/keysRSA/publickey1248
config1/keysRSA/publickey7002 config1/keysRSA/publickey1249
config1/keysRSA/publickey7002 config1/keysRSA/publickey1250
config1/keysRSA/publickey7002 config1/keysRSA/publickey1251
config1/keysRSA/publickey7002 config1/keysRSA/publickey1252
config1/keysRSA/publickey7002 config1/keysRSA/publickey1253
config1/keysRSA/publickey7002 config1/keysRSA/publickey1254
config1/keysRSA/publickey7002 config1/keysRSA/publickey1255
config1/keysRSA/publickey7002 config1/keysRSA/publickey1256
config1/keysRSA/publickey7002 config1/keysRSA/publickey1257
config1/keysRSA/publickey7002 config1/keysRSA/publickey1258
config1/keysRSA/publickey7002 config1/ke

config2/keysRSA/publickey7002 config2/keysRSA/publickey1008
config2/keysRSA/publickey7002 config2/keysRSA/publickey1009
config2/keysRSA/publickey7002 config2/keysRSA/publickey1010
config2/keysRSA/publickey7002 config2/keysRSA/publickey1011
config2/keysRSA/publickey7002 config2/keysRSA/publickey1012
config2/keysRSA/publickey7002 config2/keysRSA/publickey1013
config2/keysRSA/publickey7002 config2/keysRSA/publickey1014
config2/keysRSA/publickey7002 config2/keysRSA/publickey1015
config2/keysRSA/publickey7002 config2/keysRSA/publickey1016
config2/keysRSA/publickey7002 config2/keysRSA/publickey1017
config2/keysRSA/publickey7002 config2/keysRSA/publickey1018
config2/keysRSA/publickey7002 config2/keysRSA/publickey1019
config2/keysRSA/publickey7002 config2/keysRSA/publickey1020
config2/keysRSA/publickey7002 config2/keysRSA/publickey1021
config2/keysRSA/publickey7002 config2/keysRSA/publickey1022
config2/keysRSA/publickey7002 config2/keysRSA/publickey1023
config2/keysRSA/publickey7002 config2/ke

...

config9/keysRSA/publickey7002 config9/keysRSA/publickey1034
config9/keysRSA/publickey7002 config9/keysRSA/publickey1035
config9/keysRSA/publickey7002 config9/keysRSA/publickey1036
config9/keysRSA/publickey7002 config9/keysRSA/publickey1037
config9/keysRSA/publickey7002 config9/keysRSA/publickey1038
config9/keysRSA/publickey7002 config9/keysRSA/publickey1039
config9/keysRSA/publickey7002 config9/keysRSA/publickey1040
config9/keysRSA/publickey7002 config9/keysRSA/publickey1041
config9/keysRSA/publickey7002 config9/keysRSA/publickey1042
config9/keysRSA/publickey7002 config9/keysRSA/publickey1043
config9/keysRSA/publickey7002 config9/keysRSA/publickey1044
config9/keysRSA/publickey7002 config9/keysRSA/publickey1045
config9/keysRSA/publickey7002 config9/keysRSA/publickey1046
config9/keysRSA/publickey7002 config9/keysRSA/publickey1047
config9/keysRSA/publickey7002 config9/keysRSA/publickey1048
config9/keysRSA/publickey7002 config9/keysRSA/publickey1049
config9/keysRSA/publickey7002 config9/ke

config9/keysRSA/publickey7002 config9/keysRSA/publickey1276
config9/keysRSA/publickey7002 config9/keysRSA/publickey1277
config9/keysRSA/publickey7002 config9/keysRSA/publickey1278
config9/keysRSA/publickey7002 config9/keysRSA/publickey1279
config9/keysRSA/publickey7002 config9/keysRSA/publickey1280
config9/keysRSA/publickey7002 config9/keysRSA/publickey1281
config9/keysRSA/publickey7002 config9/keysRSA/publickey1282
config9/keysRSA/publickey7002 config9/keysRSA/publickey1283
config9/keysRSA/publickey7002 config9/keysRSA/publickey1284
config9/keysRSA/publickey7002 config9/keysRSA/publickey1285
config9/keysRSA/publickey7002 config9/keysRSA/publickey1286
config9/keysRSA/publickey7002 config9/keysRSA/publickey1287
config9/keysRSA/publickey7002 config9/keysRSA/publickey1288
config9/keysRSA/publickey7002 config9/keysRSA/publickey1289
config9/keysRSA/publickey7002 config9/keysRSA/publickey1290
config9/keysRSA/publickey7002 config9/keysRSA/publickey1291
config9/keysRSA/publickey7002 config9/ke

config10/keysRSA/publickey7002 config10/keysRSA/publickey1150
config10/keysRSA/publickey7002 config10/keysRSA/publickey1151
config10/keysRSA/publickey7002 config10/keysRSA/publickey1152
config10/keysRSA/publickey7002 config10/keysRSA/publickey1153
config10/keysRSA/publickey7002 config10/keysRSA/publickey1154
config10/keysRSA/publickey7002 config10/keysRSA/publickey1155
config10/keysRSA/publickey7002 config10/keysRSA/publickey1156
config10/keysRSA/publickey7002 config10/keysRSA/publickey1157
config10/keysRSA/publickey7002 config10/keysRSA/publickey1158
config10/keysRSA/publickey7002 config10/keysRSA/publickey1159
config10/keysRSA/publickey7002 config10/keysRSA/publickey1160
config10/keysRSA/publickey7002 config10/keysRSA/publickey1161
config10/keysRSA/publickey7002 config10/keysRSA/publickey1162
config10/keysRSA/publickey7002 config10/keysRSA/publickey1163
config10/keysRSA/publickey7002 config10/keysRSA/publickey1164
config10/keysRSA/publickey7002 config10/keysRSA/publickey1165
config10

config10/keysRSA/publickey7002 config10/keysRSA/publickey1328
config10/keysRSA/publickey7002 config10/keysRSA/publickey1329
config10/keysRSA/publickey7002 config10/keysRSA/publickey1330
config10/keysRSA/publickey7002 config10/keysRSA/publickey1331
config10/keysRSA/publickey7002 config10/keysRSA/publickey1332
config10/keysRSA/publickey7002 config10/keysRSA/publickey1333
config10/keysRSA/publickey7002 config10/keysRSA/publickey1334
config10/keysRSA/publickey7002 config10/keysRSA/publickey1335
config10/keysRSA/publickey7002 config10/keysRSA/publickey1336
config10/keysRSA/publickey7002 config10/keysRSA/publickey1337
config10/keysRSA/publickey7002 config10/keysRSA/publickey1338
config10/keysRSA/publickey7002 config10/keysRSA/publickey1339
config10/keysRSA/publickey7002 config10/keysRSA/publickey1340
config10/keysRSA/publickey7002 config10/keysRSA/publickey1341
config10/keysRSA/publickey7002 config10/keysRSA/publickey1342
config10/keysRSA/publickey7002 config10/keysRSA/publickey1343
config10

config11/keysRSA/publickey7002 config11/keysRSA/publickey1216
config11/keysRSA/publickey7002 config11/keysRSA/publickey1217
config11/keysRSA/publickey7002 config11/keysRSA/publickey1218
config11/keysRSA/publickey7002 config11/keysRSA/publickey1219
config11/keysRSA/publickey7002 config11/keysRSA/publickey1220
config11/keysRSA/publickey7002 config11/keysRSA/publickey1221
config11/keysRSA/publickey7002 config11/keysRSA/publickey1222
config11/keysRSA/publickey7002 config11/keysRSA/publickey1223
config11/keysRSA/publickey7002 config11/keysRSA/publickey1224
config11/keysRSA/publickey7002 config11/keysRSA/publickey1225
config11/keysRSA/publickey7002 config11/keysRSA/publickey1226
config11/keysRSA/publickey7002 config11/keysRSA/publickey1227
config11/keysRSA/publickey7002 config11/keysRSA/publickey1228
config11/keysRSA/publickey7002 config11/keysRSA/publickey1229
config11/keysRSA/publickey7002 config11/keysRSA/publickey1230
config11/keysRSA/publickey7002 config11/keysRSA/publickey1231
config11

In [20]:
# os.system('git remote set-url origin https://github.com/tmane002/BftSmart_Final.git')

In [21]:
os.system('git add .; git commit -m "Testing";git push')

[main e906183] Testing
 14192 files changed, 24860 insertions(+), 7625 deletions(-)
 rewrite PrepConfig.ipynb (78%)
 rewrite all_external_ips (100%)
 rewrite all_internal_ips (100%)
 rewrite config/keys/privatekey0 (100%)
 rewrite config/keys/privatekey1 (100%)
 rewrite config/keys/privatekey10 (100%)
 rewrite config/keys/privatekey11 (100%)
 rewrite config/keys/privatekey12 (100%)
 rewrite config/keys/privatekey13 (100%)
 rewrite config/keys/privatekey14 (100%)
 rewrite config/keys/privatekey15 (100%)
 rewrite config/keys/privatekey16 (100%)
 rewrite config/keys/privatekey17 (100%)
 rewrite config/keys/privatekey18 (100%)
 rewrite config/keys/privatekey19 (100%)
 rewrite config/keys/privatekey2 (100%)
 rewrite config/keys/privatekey20 (100%)
 rewrite config/keys/privatekey21 (100%)
 rewrite config/keys/privatekey22 (100%)
 rewrite config/keys/privatekey23 (100%)
 rewrite config/keys/privatekey24 (100%)
 rewrite config/keys/privatekey25 (100%)
 rewrite config/keys/privatekey26 (100%)
 

To github.com:tmane002/BftSmart_Final.git
   235dc6e..e906183  main -> main


0

In [22]:
# 'gcloud compute instances create instance-1 --project=cellular-hybrid-358402 --zone=us-central1-a --machine-type=e2-medium --network-interface=network-tier=PREMIUM,subnet=default --no-restart-on-failure --maintenance-policy=TERMINATE --provisioning-model=SPOT --instance-termination-action=STOP --service-account=932709639592-compute@developer.gserviceaccount.com --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append --tags=http-server,https-server --create-disk=auto-delete=yes,boot=yes,device-name=instance-1,image=projects/ubuntu-os-cloud/global/images/ubuntu-2204-jammy-v20220810,mode=rw,size=10,type=projects/cellular-hybrid-358402/zones/us-central1-a/diskTypes/pd-balanced --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --reservation-affinity=any'

In [23]:
# gcloud compute instances create instance-1 --project=cellular-hybrid-358402 --zone=us-west1-b --machine-type=e2-medium --network-interface=network-tier=PREMIUM,subnet=default --no-restart-on-failure --maintenance-policy=TERMINATE --provisioning-model=SPOT --instance-termination-action=STOP --service-account=932709639592-compute@developer.gserviceaccount.com --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append --tags=http-server,https-server --create-disk=auto-delete=yes,boot=yes,device-name=instance-1,image=projects/ubuntu-os-cloud/global/images/ubuntu-2204-jammy-v20220810,mode=rw,size=10,type=projects/cellular-hybrid-358402/zones/us-central1-a/diskTypes/pd-balanced --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --reservation-affinity=any